### Leer todos los datos que son requeridos

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
movies_df = spark.read.format("delta").load(f"{silver_folder_path}/movies").filter(f"file_date = '{v_file_date}'")

In [0]:
countries_df = spark.read.format("delta").load(f"{silver_folder_path}/countries")

In [0]:
productions_countries_df = spark.read.format("delta").load(f"{silver_folder_path}/productions_countries").filter(f"file_date = '{v_file_date}'")

### Join entre "countries" y "movies_countries" 

In [0]:
countries_prod_coun_df = countries_df.join(productions_countries_df, on='country_id', how='inner').select('country_name', "movie_id")

### Join entre "movies_df" y "countries_prod_coun_df"

- Filtrar las peliculas donde su fecha de lanzamiento sea mayor o igual a 2015.

In [0]:
movie_filter_df = movies_df.filter(movies_df.year_release_date >= 2015)

In [0]:
results_movies_countries_df = movie_filter_df.join(countries_prod_coun_df, on='movie_id', how='inner')

In [0]:
results_df = results_movies_countries_df.select("year_release_date","budget","revenue","country_name")

In [0]:
from pyspark.sql.functions import lit

In [0]:
from pyspark.sql.functions import sum
results_group_by_df = results_df.groupBy("year_release_date","country_name").agg(sum("budget").alias("total_budget"),sum("revenue").alias("total_revenue"))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, desc, asc
results_dense_rank_df = Window.partitionBy("year_release_date").orderBy(desc("total_budget"),desc("total_revenue"))
final_df = results_group_by_df.withColumn("dense_rank",dense_rank().over(results_dense_rank_df)).withColumn("created_date",lit(v_file_date))

### Escribir datos en el DataLake en formato "Delta"

In [0]:
#ovewrite_partition(final_df,"movie_gold","results_group_movie_country","created_date")

In [0]:
merge_condition = 'tgt.year_release_date = src.year_release_date AND tgt.country_name = src.country_name AND tgt.created_date = src.created_date'
merge_delta_lake(final_df, 'movie_gold', 'results_group_movie_country', gold_folder_path, merge_condition, 'created_date')

In [0]:
%sql
SELECT *
FROM movie_gold.results_group_movie_country;